!pip install seaborn

In [20]:
from __future__ import absolute_import, division, print_function

import pathlib

import pandas as pd
import numpy as np
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

np.random.seed(2)

2.3.0


In [21]:
dataset = pd.read_csv('/content/diabetes_with_na.csv')
dataset.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,NaN,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,NaN,NaN,30.1,0.349,47,1
767,1,93.0,70.0,31.0,NaN,30.4,0.315,23,0


In [22]:
dataset.isna().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

train_stats = train_dataset.describe()
train_stats.pop("Outcome")
train_stats = train_stats.transpose()
train_stats

In [24]:
def find_median(var):
    temp = dataset[dataset[var].notnull()]
    temp = dataset[[var,'Outcome']].groupby('Outcome')[[var]].median().reset_index()
    return temp

In [25]:
find_median('Insulin')

,Outcome,Insulin
0,0,102.5
1,1,169.5


In [26]:
find_median('SkinThickness')

,Outcome,SkinThickness
0,0,27.0
1,1,32.0


In [27]:
find_median('Glucose')

,Outcome,Glucose
0,0,107.0
1,1,140.0


In [28]:
find_median('BloodPressure')

,Outcome,BloodPressure
0,0,70.0
1,1,74.5


In [29]:
find_median('BMI')

,Outcome,BMI
0,0,30.1
1,1,34.3


In [30]:
dataset.loc[(dataset['Outcome'] == 0) & (dataset['Insulin'].isna()), 'Insulin'] = 102.5
dataset.loc[(dataset['Outcome'] == 1) & (dataset['Insulin'].isna()), 'Insulin'] = 169.5

dataset.loc[(dataset['Outcome'] == 0) & (dataset['SkinThickness'].isna()), 'SkinThickness'] = 27.0
dataset.loc[(dataset['Outcome'] == 1) & (dataset['SkinThickness'].isna()), 'SkinThickness'] = 32.0

dataset.loc[(dataset['Outcome'] == 0) & (dataset['Glucose'].isna()), 'Glucose'] = 107.0
dataset.loc[(dataset['Outcome'] == 1) & (dataset['Glucose'].isna()), 'Glucose'] = 140.0

dataset.loc[(dataset['Outcome'] == 0) & (dataset['BloodPressure'].isna()), 'BloodPressure'] = 70.0
dataset.loc[(dataset['Outcome'] == 1) & (dataset['BloodPressure'].isna()), 'BloodPressure'] = 74.0

dataset.loc[(dataset['Outcome'] == 0) & (dataset['BMI'].isna()), 'BMI'] = 30.1
dataset.loc[(dataset['Outcome'] == 1) & (dataset['BMI'].isna()), 'BMI'] = 34.3

In [31]:
dataset.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [32]:
from mlxtend.preprocessing import minmax_scaling

scaled_data = minmax_scaling(dataset,columns=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'])

In [33]:
def build_model():
  model = keras.Sequential([
                            layers.Dense(15,activation=tf.nn.relu, input_shape=[len(scaled_data.keys())]),
                            layers.Dense(10, activation=tf.nn.relu),
                            layers.Dense(8, activation=tf.nn.relu),
                            layers.Dense(1,activation=tf.nn.sigmoid)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])
  return model

model = build_model()

In [35]:
model = build_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 15)                135       
_________________________________________________________________
dense_17 (Dense)             (None, 10)                160       
_________________________________________________________________
dense_18 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 9         
Total params: 392
Trainable params: 392
Non-trainable params: 0
_________________________________________________________________


In [36]:
EPOCHS = 1000

history = model.fit(scaled_data, dataset['Outcome'], epochs=EPOCHS, validation_split=0.2, verbose=2)

Epoch 1/1000
20/20 - 0s - loss: 0.7139 - accuracy: 0.3502 - val_loss: 0.6952 - val_accuracy: 0.4091
Epoch 2/1000
20/20 - 0s - loss: 0.6889 - accuracy: 0.6026 - val_loss: 0.6847 - val_accuracy: 0.7013
Epoch 3/1000
20/20 - 0s - loss: 0.6809 - accuracy: 0.7427 - val_loss: 0.6797 - val_accuracy: 0.6688
Epoch 4/1000
20/20 - 0s - loss: 0.6754 - accuracy: 0.7003 - val_loss: 0.6747 - val_accuracy: 0.6818
Epoch 5/1000
20/20 - 0s - loss: 0.6702 - accuracy: 0.6938 - val_loss: 0.6706 - val_accuracy: 0.7208
Epoch 6/1000
20/20 - 0s - loss: 0.6647 - accuracy: 0.7182 - val_loss: 0.6645 - val_accuracy: 0.7078
Epoch 7/1000
20/20 - 0s - loss: 0.6597 - accuracy: 0.7036 - val_loss: 0.6590 - val_accuracy: 0.7013
Epoch 8/1000
20/20 - 0s - loss: 0.6531 - accuracy: 0.7117 - val_loss: 0.6520 - val_accuracy: 0.7338
Epoch 9/1000
20/20 - 0s - loss: 0.6467 - accuracy: 0.7231 - val_loss: 0.6455 - val_accuracy: 0.6688
Epoch 10/1000
20/20 - 0s - loss: 0.6395 - accuracy: 0.7248 - val_loss: 0.6391 - val_accuracy: 0.6688

In [37]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail

<bound method NDFrame.tail of          loss  accuracy  val_loss  val_accuracy  epoch
0    0.713861  0.350163  0.695242      0.409091      0
1    0.688946  0.602606  0.684723      0.701299      1
2    0.680883  0.742671  0.679701      0.668831      2
3    0.675401  0.700326  0.674655      0.681818      3
4    0.670241  0.693811  0.670594      0.720779      4
..        ...       ...       ...           ...    ...
995  0.214275  0.905537  0.364358      0.857143    995
996  0.218168  0.905537  0.374368      0.863636    996
997  0.217254  0.908795  0.353860      0.837662    997
998  0.216372  0.910423  0.375705      0.857143    998
999  0.214438  0.912052  0.397265      0.857143    999

[1000 rows x 5 columns]>

In [38]:
acc = (hist['accuracy'].tail().sum())*100/5 
val_acc = (hist['val_accuracy'].tail().sum())*100/5 

print("Training Accuracy = {}% and Validation Accuracy= {}%".format(acc,val_acc))

Training Accuracy = 90.84690570831299% and Validation Accuracy= 85.45454621315002%
